In [ ]:
from tensorflow.keras.layers import Masking, Input, Conv2D, MaxPooling2D, Flatten, Dense, Multiply, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
import tensorflow as tf
# Input shape based on the court grid and assuming a single channel for simplicity
input_shape = (n_rows, n_cols, 1)  # n_rows=94, n_cols=50, channels=1 for player positions

# Model architecture
inputs = Input(shape=input_shape)
mask = Masking(mask_value=0)(inputs)  # Masking layer to ignore padded zeros
# Base convolutional layers with fewer neurons
conv1 = Conv2D(16, kernel_size=(3, 3), activation='relu')(mask)  # Reduced from 32 to 16
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(32, (3, 3), activation='relu')(pool1)  # Reduced from 64 to 32
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

# Spatial attention layer remains the same
attention_probs = Conv2D(1, kernel_size=(1, 1), activation='sigmoid')(pool2)
attention_mul = Multiply()([pool2, attention_probs])

# Flattening and dense layers with fewer neurons
flatten = Flatten()(attention_mul)
dense1 = Dense(64, activation='relu')(flatten)  # Reduced from 128 to 64
dropout = Dropout(0.5)(dense1)
output = Dense(2, activation='softmax')(dropout)  # Output layer remains the same for binary classification

# Construct and compile the model
model = Model(inputs=[inputs], outputs=[output])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary to inspect the architecture
model.summary()